In [1]:
import digitalhub as dh
PROJECT_NAME = "landslide-monitoring"
proj = dh.get_or_create_project(PROJECT_NAME)

Register to the open data space copenicus(if not already) and get your credentials.

https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=FIiRPJeoiX4

Log the credentials as project secret keys as shown below

In [2]:
# THIS NEED TO BE EXECUTED JUST ONCE
secret0 = proj.new_secret(name="CDSETOOL_ESA_USER", secret_value="esa_username")
secret1 = proj.new_secret(name="CDSETOOL_ESA_PASSWORD", secret_value="esa_password")

### Download data from Sentinel 1 (Orbit Direction Ascending)

In [ ]:
function_s1 = proj.new_function("download_images_s1",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.11.2",command="python")

In [ ]:
string_dict_data = """{
  'satelliteParams':{
      'satelliteType': 'Sentinel1',
      'processingLevel': 'LEVEL1',
      'sensorMode': 'IW',
      'productType': 'SLC',
      'orbitDirection': 'ASCENDING',
      'relativeOrbitNumber': '168'
  } ,
  'startDate': '2021-03-01',
  'endDate': '2023-03-31',
  'geometry': 'POLYGON((10.81295 45.895743, 10.813637 45.895743, 10.813637 45.89634, 10.81295 45.89634, 10.81295 45.895743))',
  'area_sampling': 'True',
  'tmp_path_same_folder_dwl':'True',
  'artifact_name': 's1_ascending'
  }"""
list_args =  ["main.py",string_dict_data]

In [7]:
run = function_s1.run(
    action="job",
    secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
    fs_group='8877',
    args=["main.py", string_dict_data],
    resources={"cpu": {"requests": "3", "limits": "6"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-land",
        "mount_path": "/app/files",
        "spec": {
             "size": "100Gi" 
        }
    }])

### Download data from Sentinel 1 (Orbit Direction Descending)

In [ ]:
string_dict_data = """{
  'satelliteParams':{
      'satelliteType': 'Sentinel1',
      'processingLevel': 'LEVEL1',
      'sensorMode': 'IW',
      'productType': 'SLC',
      'orbitDirection': 'DESCENDING',
      'relativeOrbitNumber': '168'
  } ,
  'startDate': '2021-03-01',
  'endDate': '2023-03-31',
  'geometry': 'POLYGON((10.81295 45.895743, 10.813637 45.895743, 10.813637 45.89634, 10.81295 45.89634, 10.81295 45.895743))',
  'area_sampling': 'True',
  'tmp_path_same_folder_dwl':'True',
  'artifact_name': 's1_descending'
  }"""
list_args =  ["main.py",string_dict_data]

In [11]:
run = function_s1.run(
    action="job",
    secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
    fs_group='8877',
    args=["main.py", string_dict_data],
    resources={"cpu": {"requests": "3", "limits": "6"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-land",
        "mount_path": "/app/files",
        "spec": {
             "size": "100Gi" 
        }
    }])

### Perform Data Elaboration on RS-Tool

In [4]:
artifact_name='bosco'
src_path='bosco'
artifact_bosco = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

In [4]:
# artifact_name='data'
# src_path='data'
# artifact_data = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

In [5]:
function_rs = proj.new_function("elaborate",kind="container", image="ghcr.io/tn-aixpa/rs-deforestation:2.6_b8", command="/bin/bash", code_src="launch.sh")

In [ ]:
#function_rs = proj.get_function("elaborate")

In [ ]:
run_el = function_rs.run(
    action="job",
    fs_group='8877',
    resources={"cpu": {"requests": "6", "limits": "12"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-deforestation",
        "mount_path": "/app/files",
        "spec": { "size": "250Gi" }
    }],
    args=['/shared/launch.sh', 'bosco', 'data_s2_deforestation', '[2018,2019]',  'deforestation_2018_19']
)

In [11]:
run_el.refresh().status.state

'RUNNING'

### Pipeline

In [2]:
%%writefile "deforestation_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(startYear, endYear, geometry, shapeArtifactName, dataArtifactName, outputName):
    with pipeline_context() as pc:
        string_dict_data = """{"satelliteParams":{"satelliteType":"Sentinel2"},"startDate":\""""+ str(startYear) + """-01-01\","endDate": \"""" + str(endYear) + """-12-31\","geometry": \"""" + str(geometry) + """\","area_sampling":"true","cloudCover":"[0,20]","artifact_name":"data_s2_deforestation"}"""
        s1 = pc.step(name="download",
                     function="download_images_s2",
                     action="job",
                     secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
                     fs_group='8877',
                     args=["main.py", string_dict_data],
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-deforestation",
                        "mount_path": "/app/files",
                        "spec": { "size": "250Gi" }
                        }
                    ])
        s2 = pc.step(name="elaborate",
                     function="elaborate",
                     action="job",
                     fs_group='8877',
                     resources={"cpu": {"requests": "6", "limits": "12"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-deforestation",
                        "mount_path": "/app/files",
                        "spec": { "size": "250Gi" }
                    }],
                     args=['/shared/launch.sh', str(shapeArtifactName), 'data_s2_deforestation', "[" + str(startYear) + ',' + str(endYear) + "]", str(outputName)]
                     ).after(s1)
     


Overwriting deforestation_pipeline.py


In [3]:
workflow = proj.new_workflow(name="pipeline_deforestation", kind="kfp", code_src= "deforestation_pipeline.py", handler = "myhandler")

In [5]:
wfbuild = workflow.run(action="build", wait=True)

2025-06-10 11:19:53,408 - INFO - Waiting for run 66e5b16b2d5340b398f6fe5ca872e32e to finish...
2025-06-10 11:19:58,436 - INFO - Waiting for run 66e5b16b2d5340b398f6fe5ca872e32e to finish...
2025-06-10 11:20:03,689 - INFO - Waiting for run 66e5b16b2d5340b398f6fe5ca872e32e to finish...
2025-06-10 11:20:08,723 - INFO - Run 66e5b16b2d5340b398f6fe5ca872e32e finished in 15.32 seconds.


In [7]:
workflow_run = workflow.run(action="pipeline", parameters={
    "startYear": 2018,
    "endYear": 2019,
    "geometry": "POLYGON ((10.71297105232361 46.091369896369656, 10.71297105232361 45.91562543985691, 11.096755162591023 45.91562543985691, 11.096755162591023 46.091369896369656, 10.71297105232361 46.091369896369656))",
    "shapeArtifactName":"bosco",
    'dataArtifactName': "data_s2_deforestation",
    "outputName": "deforestation_2018_19"
    })